# Insurance cost prediction using linear regression

Make a submisson here: https://jovian.ai/learn/deep-learning-with-pytorch-zero-to-gans/assignment/assignment-2-train-your-first-model

In this assignment we're going to use information like a person's age, sex, BMI, no. of children and smoking habit to predict the price of yearly medical bills. This kind of model is useful for insurance companies to determine the yearly insurance premium for a person. The dataset for this problem is taken from [Kaggle](https://www.kaggle.com/mirichoi0218/insurance).


We will create a model with the following steps:
1. Download and explore the dataset
2. Prepare the dataset for training
3. Create a linear regression model
4. Train the model to fit the data
5. Make predictions using the trained model


This assignment builds upon the concepts from the first 2 lessons. It will help to review these Jupyter notebooks:
- PyTorch basics: https://jovian.ai/aakashns/01-pytorch-basics
- Linear Regression: https://jovian.ai/aakashns/02-linear-regression
- Logistic Regression: https://jovian.ai/aakashns/03-logistic-regression
- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

As you go through this notebook, you will find a **???** in certain places. Your job is to replace the **???** with appropriate code or values, to ensure that the notebook runs properly end-to-end . In some cases, you'll be required to choose some hyperparameters (learning rate, batch size etc.). Try to experiment with the hypeparameters to get the lowest loss.


In [1]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows
# !pip install numpy matplotlib pandas torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy matplotlib pandas torch torchvision torchaudio

In [2]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [3]:
torch.set_default_dtype(torch.float64)

In [4]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [5]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [6]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [7]:
your_name = 'jorgais' # at least 5 characters

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [8]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [9]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,charges
619,55,female,41.18100,0,no,12213.554160
319,32,male,41.44185,1,no,5321.072721
34,28,male,40.40400,1,yes,58361.797420
403,49,male,35.85300,3,no,11707.184400
113,21,female,39.64920,0,no,2741.396532


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [10]:
num_rows = dataframe.shape[0]  #???
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [11]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1271 entries, 619 to 1036
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1271 non-null   int64  
 1   sex       1271 non-null   object 
 2   bmi       1271 non-null   float64
 3   children  1271 non-null   int64  
 4   smoker    1271 non-null   object 
 5   charges   1271 non-null   float64
dtypes: float64(2), int64(2), object(2)
memory usage: 69.5+ KB


In [12]:
num_cols = dataframe.shape[1] #???
print(num_cols)

6


**Q: What are the column titles of the input variables?**

In [13]:
input_cols = ['age','bmi','sex','smoker','children',]
input_cols

['age', 'bmi', 'sex', 'smoker', 'children']

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [14]:

categorical_cols = ['sex', 'smoker']
categorical_cols


['sex', 'smoker']

**Q: What are the column titles of output/target variable(s)?**

In [15]:
output_cols = ['charges']
output_cols

['charges']

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [16]:
# Write your answer here

Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [17]:
!pip install jovian --upgrade -q

In [18]:
import jovian

In [19]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Updating notebook "tjorgais002/02-insurance-linear-regression" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/tjorgais002/02-insurance-linear-regression


'https://jovian.ai/tjorgais002/02-insurance-linear-regression'

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [20]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [21]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[55.     , 41.181  ,  0.     ,  0.     ,  0.     ],
        [32.     , 41.44185,  1.     ,  0.     ,  1.     ],
        [28.     , 40.404  ,  1.     ,  1.     ,  1.     ],
        ...,
        [54.     , 32.412  ,  1.     ,  0.     ,  1.     ],
        [18.     , 30.3696 ,  1.     ,  1.     ,  1.     ],
        [22.     , 41.1477 ,  1.     ,  1.     ,  2.     ]]),
 array([[12213.55416  ],
        [ 5321.072721 ],
        [58361.7974196],
        ...,
        [11897.14944  ],
        [19583.697936 ],
        [42732.272202 ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [22]:
inputs = torch.from_numpy(inputs_array)
targets = torch.from_numpy(targets_array)

In [23]:
inputs.dtype, targets.dtype

(torch.float64, torch.float64)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [24]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets.**

In [25]:
val_percent = 0.2 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size, val_size]) # Use the random_split function to split dataset into 2 parts of the desired length

In [26]:
print(val_size)
print(train_size)

254
1017


Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [27]:
batch_size = 50

In [28]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [29]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[33.0000, 39.6825,  1.0000,  1.0000,  1.0000],
        [62.0000, 36.8520,  0.0000,  0.0000,  0.0000],
        [61.0000, 35.0427,  1.0000,  0.0000,  0.0000],
        [63.0000, 27.8388,  0.0000,  0.0000,  0.0000],
        [54.0000, 35.4090,  0.0000,  0.0000,  1.0000],
        [47.0000, 26.7510,  0.0000,  0.0000,  1.0000],
        [18.0000, 25.7631,  1.0000,  0.0000,  0.0000],
        [47.0000, 50.3052,  0.0000,  0.0000,  1.0000],
        [24.0000, 37.3293,  1.0000,  0.0000,  4.0000],
        [48.0000, 33.5220,  1.0000,  0.0000,  2.0000],
        [55.0000, 32.1623,  1.0000,  0.0000,  0.0000],
        [24.0000, 29.5260,  0.0000,  0.0000,  0.0000],
        [36.0000, 33.3222,  0.0000,  0.0000,  0.0000],
        [53.0000, 29.5260,  0.0000,  0.0000,  0.0000],
        [19.0000, 22.9770,  1.0000,  0.0000,  0.0000],
        [52.0000, 25.7298,  0.0000,  0.0000,  0.0000],
        [63.0000, 40.2930,  0.0000,  0.0000,  0.0000],
        [26.0000, 26.3070,  1.0000,  0.0000,  2.0000],
  

Let's save our work by committing to Jovian.

In [30]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "tjorgais002/02-insurance-linear-regression" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/tjorgais002/02-insurance-linear-regression


'https://jovian.ai/tjorgais002/02-insurance-linear-regression'

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [31]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [32]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)            # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        #xb=xb.type(torch.FloatTensor)
        out = self.linear(xb)                          # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.mse_loss(out, targets)                          # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.mse_loss(out,targets)                          # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [33]:

model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [34]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2866,  0.4144, -0.0585,  0.0845, -0.3871]], requires_grad=True),
 Parameter containing:
 tensor([0.0927], requires_grad=True)]

One final commit before we train the model.

In [35]:
jovian.commit(project=project_name, environment=None)

<IPython.core.display.Javascript object>

[jovian] Updating notebook "tjorgais002/02-insurance-linear-regression" on https://jovian.ai
[jovian] Committed successfully! https://jovian.ai/tjorgais002/02-insurance-linear-regression


'https://jovian.ai/tjorgais002/02-insurance-linear-regression'

## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [36]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [37]:
result = evaluate(model, val_loader) # Use the the evaluate function
print(result)

{'val_loss': 351761445.7011819}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [38]:
epochs = 100
lr = 0.0003
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 199602152.8304
Epoch [40], val_loss: 134048965.0859
Epoch [60], val_loss: 161876896.7403
Epoch [80], val_loss: 119597281.9426
Epoch [100], val_loss: 277608922.8076


In [39]:
epochs = 200
lr = 0.0002
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 152745656.8354
Epoch [40], val_loss: 121664528.0690
Epoch [60], val_loss: 97987542.0942
Epoch [80], val_loss: 108019011.1244
Epoch [100], val_loss: 131700447.2379
Epoch [120], val_loss: 92713916.6206
Epoch [140], val_loss: 131391677.3147
Epoch [160], val_loss: 109098681.4053
Epoch [180], val_loss: 80256896.9087
Epoch [200], val_loss: 101400291.5170


In [40]:
epochs = 300
lr = 0.0002
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 133578575.2137
Epoch [40], val_loss: 102677708.2513
Epoch [60], val_loss: 73420969.3277
Epoch [80], val_loss: 73549006.8522
Epoch [100], val_loss: 69640171.2663
Epoch [120], val_loss: 68577633.2888
Epoch [140], val_loss: 78856237.0563
Epoch [160], val_loss: 65052327.9769
Epoch [180], val_loss: 75058952.2535
Epoch [200], val_loss: 63042696.8340
Epoch [220], val_loss: 71245357.8198
Epoch [240], val_loss: 60740503.6578
Epoch [260], val_loss: 64838101.0982
Epoch [280], val_loss: 62890531.1747
Epoch [300], val_loss: 59562521.3529


In [41]:
epochs = 1000
lr = 0.0001
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 57556981.7818
Epoch [40], val_loss: 57571798.8958
Epoch [60], val_loss: 66150972.6934
Epoch [80], val_loss: 68060304.4062
Epoch [100], val_loss: 56493324.8277
Epoch [120], val_loss: 67849523.7475
Epoch [140], val_loss: 60719747.3651
Epoch [160], val_loss: 55168004.0052
Epoch [180], val_loss: 58436530.3059
Epoch [200], val_loss: 54335375.6197
Epoch [220], val_loss: 54172011.2533
Epoch [240], val_loss: 53824834.6699
Epoch [260], val_loss: 53332466.0269
Epoch [280], val_loss: 56430824.6994
Epoch [300], val_loss: 56100176.4102
Epoch [320], val_loss: 52384297.3735
Epoch [340], val_loss: 52238378.3008
Epoch [360], val_loss: 52993741.3703
Epoch [380], val_loss: 54570193.9959
Epoch [400], val_loss: 52481299.9584
Epoch [420], val_loss: 57011929.0433
Epoch [440], val_loss: 51288456.8821
Epoch [460], val_loss: 52849992.6914
Epoch [480], val_loss: 54344589.6856
Epoch [500], val_loss: 53051203.2487
Epoch [520], val_loss: 51092428.9775
Epoch [540], val_loss: 50291513.7697
Epoch

In [42]:
epochs = 1000
lr = 0.0001
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 49227729.0771
Epoch [40], val_loss: 46827270.9129
Epoch [60], val_loss: 46978160.7157
Epoch [80], val_loss: 46712317.9740
Epoch [100], val_loss: 47492011.1874
Epoch [120], val_loss: 48978399.1148
Epoch [140], val_loss: 48868064.6324
Epoch [160], val_loss: 55468699.9711
Epoch [180], val_loss: 49729840.1037
Epoch [200], val_loss: 48444465.5776
Epoch [220], val_loss: 47539229.9694
Epoch [240], val_loss: 46215082.2635
Epoch [260], val_loss: 46370972.7058
Epoch [280], val_loss: 54659484.6622
Epoch [300], val_loss: 46112817.9182
Epoch [320], val_loss: 45909908.8286
Epoch [340], val_loss: 53183888.8723
Epoch [360], val_loss: 46428630.7686
Epoch [380], val_loss: 46785021.4772
Epoch [400], val_loss: 48257671.9658
Epoch [420], val_loss: 45962247.4766
Epoch [440], val_loss: 46263203.5301
Epoch [460], val_loss: 50760614.1790
Epoch [480], val_loss: 47222066.7857
Epoch [500], val_loss: 46064563.0542
Epoch [520], val_loss: 56627484.0214
Epoch [540], val_loss: 45513420.0989
Epoch

In [43]:
epochs = 1000
lr = 0.0001
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 52531574.7665
Epoch [40], val_loss: 45515862.7942
Epoch [60], val_loss: 46207147.5018
Epoch [80], val_loss: 47129299.2168
Epoch [100], val_loss: 44897059.4226
Epoch [120], val_loss: 45378740.4165
Epoch [140], val_loss: 45812252.4126
Epoch [160], val_loss: 46309059.1638
Epoch [180], val_loss: 45051740.8102
Epoch [200], val_loss: 48118799.7273
Epoch [220], val_loss: 46453565.3610
Epoch [240], val_loss: 53971468.5433
Epoch [260], val_loss: 46353662.1933
Epoch [280], val_loss: 48211560.0167
Epoch [300], val_loss: 45897399.7522
Epoch [320], val_loss: 45916663.1987
Epoch [340], val_loss: 45197922.3447
Epoch [360], val_loss: 46927159.2220
Epoch [380], val_loss: 44899199.3185
Epoch [400], val_loss: 47254403.0588
Epoch [420], val_loss: 48389429.5081
Epoch [440], val_loss: 66607634.2550
Epoch [460], val_loss: 47464735.5067
Epoch [480], val_loss: 48637405.4396
Epoch [500], val_loss: 48432899.6621
Epoch [520], val_loss: 52821978.2897
Epoch [540], val_loss: 45835522.8897
Epoch

**Q: What is the final validation loss of your model?**

In [46]:
val_loss = torch.sqrt(torch.tensor(44617857.7857))
val_loss

tensor(6679.6600)

Let's log the final validation loss to Jovian and commit the notebook

In [47]:
jovian.log_metrics(val_loss='6679.6600')

[jovian] Metrics logged.


In [48]:
jovian.commit(project='Insurance_regression', environment=None)

<IPython.core.display.Javascript object>

[jovian] Creating a new project "tjorgais002/Insurance_regression"
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/tjorgais002/insurance-regression


'https://jovian.ai/tjorgais002/insurance-regression'

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [49]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(input)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [50]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([27.0000, 25.6410,  1.0000,  0.0000,  0.0000])
Target: tensor([2831.4590])
Prediction: tensor(4284.0864)


In [51]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([27.0000, 23.8317,  0.0000,  0.0000,  0.0000])
Target: tensor([3822.9561])
Prediction: tensor(4224.7772)


In [52]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([20.0000, 30.3030,  1.0000,  1.0000,  0.0000])
Target: tensor([18505.4456])
Prediction: tensor(29019.6431)


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to replicate this notebook for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patterns in machine learning from problem-specific details.You can use one of these starer notebooks (just change the dataset):

- Linear regression (minimal): https://jovian.ai/aakashns/housing-linear-minimal
- Logistic regression (minimal): https://jovian.ai/aakashns/mnist-logistic-minimal

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ai/forum/t/linear-regression-and-logistic-regression-notebooks-and-blog-posts/14039

In [ ]:
jovian.commit(project='Insurance_regression', environment=None)
jovian.commit(project='Insurance_regression', environment=None) # try again, kaggle fails sometimes

<IPython.core.display.Javascript object>

[jovian] Updating notebook "tjorgais002/insurance-regression" on https://jovian.ai
[jovian] Attaching records (metrics, hyperparameters, dataset etc.)
[jovian] Committed successfully! https://jovian.ai/tjorgais002/insurance-regression


<IPython.core.display.Javascript object>